In [1]:
import os
import cv2
import mediapipe as mp
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "/content/drive/MyDrive/DeepLearningTubes/Train")
PROCESSED_DIR = os.path.join(BASE_DIR, "/content/drive/MyDrive/DeepLearningTubes/Preprocessed")

In [3]:
TARGET_SIZE = (384, 384)  # face size
MARGIN_RATIO = 0.15  #  padding
MAX_DIM = 1600 #downscale
MIN_DIM = 256  #upscale
USE_CENTER_FALLBACK = True  # if detect fail, use center crop
USE_FACE_ALIGNMENT = True

In [4]:
# face detection with mediapipe
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

face_detection = mp_face_detection.FaceDetection(
    model_selection=1,
    min_detection_confidence=0.5
)

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

def ensure_dir(path: str) -> None:
    if not os.path.exists(path):
        os.makedirs(path)

In [5]:
def CenterEye(landmarks, image_shape):
    h, w = image_shape[:2]

    # landmark untuk mata kiri dan mata kanan
    EYE_LEFT_IDX = [33, 133, 160, 159, 158, 157, 173, 144, 145, 153]
    EYE_RIGHT_IDX = [362, 263, 387, 386, 385, 384, 398, 373, 374, 380]

    # titik tengah mata kiri
    left_eye_points = []
    for idx in EYE_LEFT_IDX:
        landmark = landmarks.landmark[idx]
        left_eye_points.append([landmark.x * w, landmark.y * h])
    left_eye_center = np.mean(left_eye_points, axis=0)

    # titik tengah mata kanan
    right_eye_points = []
    for idx in EYE_RIGHT_IDX:
        landmark = landmarks.landmark[idx]
        right_eye_points.append([landmark.x * w, landmark.y * h])
    right_eye_center = np.mean(right_eye_points, axis=0)

    return left_eye_center, right_eye_center


def FaceAlign(image, left_eye, right_eye):
    """
    Meluruskan wajah berdasarkan posisi mata kiri dan mata kanan.
    """
    # sudut kemiringan antar mata
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Titik pusat kedua mata
    eyes_center = ((left_eye[0] + right_eye[0]) // 2,
                   (left_eye[1] + right_eye[1]) // 2)

    #  rotation
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D(eyes_center, angle, 1.0)

    # rotation
    aligned = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)

    return aligned, angle

def FaceAlignAdvanced(image, landmarks):
    h, w = image.shape[:2]

    # Get eye centers
    left_eye, right_eye = CenterEye(landmarks, image.shape)

    # Landmark hidung
    nose_tip = landmarks.landmark[1]
    nose_point = np.array([nose_tip.x * w, nose_tip.y * h])

    # Calculate the midpoint between eyes
    eyes_center = (left_eye + right_eye) / 2

    # Calculate angle for rotation
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))

    M = cv2.getRotationMatrix2D(tuple(eyes_center), angle, 1.0)

    # Rotasi gambar
    aligned = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)

    return aligned, angle

In [9]:
def DetectAlignCrop(image_path: str) -> np.ndarray | None:
    """Deteksi wajah + alignment + crop (versi print ringkas)."""

    print(f"→ {os.path.basename(image_path)}", end=" | ")

    # Load gambar
    try:
        pil_img = Image.open(image_path)
        pil_img = ImageOps.exif_transpose(pil_img)
        pil_img = pil_img.convert("RGB")
        img = np.array(pil_img)
    except:
        print("Load ERROR")
        return None

    orig_img = img.copy()
    orig_h, orig_w = orig_img.shape[:2]

    # Resize kecil → MIN_DIM
    short_side = min(orig_h, orig_w)
    if short_side < MIN_DIM:
        scale = MIN_DIM / float(short_side)
        img = cv2.resize(orig_img, (int(orig_w * scale), int(orig_h * scale)))

    # Resize besar → MAX_DIM
    h, w = img.shape[:2]
    if max(h, w) > MAX_DIM:
        scale = MAX_DIM / float(max(h, w))
        img = cv2.resize(img, (int(w * scale), int(h * scale)))

    # Alignment
    aligned_img = img.copy()
    if USE_FACE_ALIGNMENT:
        try:
            results_mesh = face_mesh.process(img)
            if results_mesh.multi_face_landmarks:
                left_eye, right_eye = CenterEye(
                    results_mesh.multi_face_landmarks[0],
                    img.shape
                )
                aligned_img, _ = FaceAlign(img, left_eye, right_eye)
                print("Align", end=" | ")
            else:
                print("No-align", end=" | ")
        except:
            print("Align-ERR", end=" | ")

    img = aligned_img
    h, w = img.shape[:2]

    # Deteksi wajah
    results = face_detection.process(img)
    if not results.detections:
        # coba original
        results = face_detection.process(orig_img)
        img = orig_img

    if not results.detections:
        print("No-face → center")
        # fallback center crop
        side = int(0.8 * min(h, w))
        cx, cy = w // 2, h // 2
        x1 = cx - side // 2
        y1 = cy - side // 2
        return img[y1:y1+side, x1:x1+side]

    # pilih face terbaik
    best_det = max(results.detections, key=lambda d: d.score[0])
    bbox = best_det.location_data.relative_bounding_box

    x = int(bbox.xmin * w)
    y = int(bbox.ymin * h)
    bw = int(bbox.width * w)
    bh = int(bbox.height * h)

    # margin
    mx = int(bw * MARGIN_RATIO)
    my = int(bh * MARGIN_RATIO)

    x1 = max(0, x - mx)
    y1 = max(0, y - my)
    x2 = min(x + bw + mx, w)
    y2 = min(y + bh + my, h)

    cropped = img[y1:y2, x1:x2]

    if cropped.size == 0:
        print("Empty → skip")
        return None

    print("OK")
    return cropped


In [10]:
def ProcessFolderPerson(person_name: str) -> None:
    """
    Memproses seluruh gambar dalam 1 folder orang.
    Output: wajah yang sudah dicrop + alignment.
    """
    data_folder = os.path.join(DATA_DIR, person_name)
    processed_folder = os.path.join(PROCESSED_DIR, person_name)
    ensure_dir(processed_folder)

    # Ambil semua file gambar
    files = [
        f for f in os.listdir(data_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png", ".webp", ".heic"))
    ]

    print(f"\n▶ Folder: {person_name} ({len(files)} files)")

    success = skipped = 0

    for idx, filename in enumerate(files, start=1):
        img_path = os.path.join(data_folder, filename)

        # Deteksi wajah + alignment + crop
        crop = DetectAlignCrop(img_path)

        if crop is None:
            # Tidak ada wajah → skip
            skipped += 1
            print(f"  [{idx}] {filename} → SKIPPED")
            continue

        # Resize menjadi ukuran final (untuk training model)
        crop = cv2.resize(crop, TARGET_SIZE)

        save_path = os.path.join(processed_folder, filename)

        # Simpan hasil crop
        if filename.lower().endswith(".png"):
            Image.fromarray(crop).save(save_path, "PNG")
        else:
            Image.fromarray(crop).save(save_path, "JPEG", quality=95)

        success += 1
        print(f"  [{idx}] {filename} → OK")

    print(f"   Processed: {success} | Skipped: {skipped}")


In [11]:
def main() -> None:
    """
    Menjalankan seluruh proses cropping untuk semua folder orang dalam folder Train/.
    """
    if not os.path.exists(DATA_DIR):
        print("❌ Folder Train tidak ditemukan.")
        return

    # Ambil daftar folder, tiap folder = 1 orang
    persons = [
        d for d in os.listdir(DATA_DIR)
        if os.path.isdir(os.path.join(DATA_DIR, d))
    ]

    if not persons:
        print("⚠ Tidak ada folder orang di dalam Train/")
        return

    print("=== Face Cropping Pipeline (MediaPipe) ===")
    print(f"Total folder orang: {len(persons)}\n")

    ensure_dir(PROCESSED_DIR)

    # Proses tiap folder orang satu per satu
    for person in persons:
        ProcessFolderPerson(person)

    print("\n✅ Semua gambar selesai diproses!")

if __name__ == "__main__":
    main()

=== Face Cropping Pipeline (MediaPipe) ===
Total folder orang: 70


▶ Folder: Zefanya Danovanta Tarigan (4 files)
→ Zefanya Danovanta Tarigan_4.jpg | No-align | OK
  [1] Zefanya Danovanta Tarigan_4.jpg → OK
→ Zefanya Danovanta Tarigan_3.jpg | Align | OK
  [2] Zefanya Danovanta Tarigan_3.jpg → OK
→ Zefanya Danovanta Tarigan_1.jpg | Align | OK
  [3] Zefanya Danovanta Tarigan_1.jpg → OK
→ Zefanya Danovanta Tarigan_2.jpg | No-align | OK
  [4] Zefanya Danovanta Tarigan_2.jpg → OK
   Processed: 4 | Skipped: 0

▶ Folder: Zaky Ahmad Makarim (4 files)
→ Zaky Ahmad Makarim_2.jpg | Align | OK
  [1] Zaky Ahmad Makarim_2.jpg → OK
→ Zaky Ahmad Makarim_4.jpg | Align | OK
  [2] Zaky Ahmad Makarim_4.jpg → OK
→ Zaky Ahmad Makarim_1.jpg | Align | OK
  [3] Zaky Ahmad Makarim_1.jpg → OK
→ Zaky Ahmad Makarim_3.jpg | Align | OK
  [4] Zaky Ahmad Makarim_3.jpg → OK
   Processed: 4 | Skipped: 0

▶ Folder: Sikah Nubuahtul Ilmi (4 files)
→ 20251121_223625 - Sikah Nubuahtul Ilmi.jpg | Align | OK
  [1] 20251121_223